In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import urllib.request

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup 
soup = BeautifulSoup(page, "html5lib") #Using the urllib.library,i put the HTML data into a variable soup
#soup.prettify() can be used to take a look at the underlying HTML code

all_tables=soup.find_all("table") 
right_table=soup.find('table', class_='wikitable sortable')
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df=df[~df.Borough.str.contains("Not assigned")]# Here i remove rows for which boroughs have not been assigned
df.head()

df.reset_index(inplace = True, drop = True)
df['Postal Code']=df['Postal Code'].str.strip('\n')
df['Borough']=df['Borough'].str.strip('\n')
df['Neighborhood']=df['Neighborhood'].str.strip('\n')
df

df1=pd.read_csv("http://cocl.us/Geospatial_data")
df1.head()
df2 = pd.merge(df,df1,on ='Postal Code', how ='inner')  
df2

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    bs4-4.9.1                  |                0           4 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         223 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  bs4                conda-forge/noarch::bs4-4.9.1-0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
bs4-4.9.1 

df2 here is the dataframe containing the borough and neighborhood data along with its respective latitude and longitude

In [2]:
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
Next i have fetched coordinates for Toronto by using geolocator and then created another dataframe df3 which contains 
Boroughs containing Toronto as part of their name.
Then i have made a map of Toronto using its coordinates

In [3]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

df3=df2[df2['Borough'].str.contains("Toronto")].reset_index(drop=True) #Boroughs containing Toronto in their name

df3.head()

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [ ]:
Next i have used a for loop to populate all the neighborhoods on the Toronto map

In [4]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
Then i have loaded my foursquare credentials

In [5]:
CLIENT_ID = 'SVXBQLDXOARVTQR2OG3JQFHBQK3PAVHOTN4TBLLMT1TC1HIG' # your Foursquare ID
CLIENT_SECRET = 'PCK52X4UC0TBKAAVPITUDTWPCCTVPB0INCUWS2HQA2RA1UM0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


In [ ]:
Next i have taken out the latitude and longitude of the first borough and used it to create  url to test

In [6]:
df3.loc[0, 'Borough']
lat=df3.loc[0, 'Latitude']
long=df3.loc[0, 'Longitude']
lat,long


(43.6542599, -79.3606359)

In [7]:
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SVXBQLDXOARVTQR2OG3JQFHBQK3PAVHOTN4TBLLMT1TC1HIG&client_secret=PCK52X4UC0TBKAAVPITUDTWPCCTVPB0INCUWS2HQA2RA1UM0&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f068759e4bb651dce60de99'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [ ]:
Then i have used the function used for extracting category of the venue following similar steps as in Newyork dataset

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Corktown Common,Park,43.655618,-79.356211
5,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
6,Impact Kitchen,Restaurant,43.656369,-79.356980
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


Then i have used the function used to populate the dataframe for venues for all the boroughs

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
df3_venues = getNearbyVenues(names=df3['Borough'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [18]:
print(df3_venues.shape)
df3_venues.head()

(1636, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [ ]:
Now i group all the dataframe by Borough. As seen here there are 4 Boroughs

In [19]:
df3_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,120,120,120,120,120,120
Downtown Toronto,1231,1231,1231,1231,1231,1231
East Toronto,124,124,124,124,124,124
West Toronto,161,161,161,161,161,161


In [20]:
print('There are {} uniques categories.'.format(len(df3_venues['Venue Category'].unique())))

There are 231 uniques categories.


Next for all the non categorical data i use one hot encoding to convert them to categorical data

In [21]:
# one hot encoding
df3_onehot = pd.get_dummies(df3_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df3_onehot['Borough'] = df3_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [df3_onehot.columns[-1]] + list(df3_onehot.columns[:-1])
df3_onehot = df3_onehot[fixed_columns]

df3_onehot.head()

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next i have grouped all the categories as per the 4 Boroughs and calculated the category mean for each Borough

In [22]:
df3_grouped = df3_onehot.groupby('Borough').mean().reset_index()
df3_grouped

,Borough,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,...,0.000000,0.008333,0.008333,0.000000,0.008333,0.000000,0.008333,0.000000,0.000000,0.008333
1,Downtown Toronto,0.000812,0.000812,0.000812,0.001625,0.001625,0.001625,0.012998,0.001625,0.004062,...,0.000812,0.000000,0.000812,0.002437,0.010561,0.001625,0.003249,0.006499,0.000812,0.005686
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,0.000000,0.000000,...,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,0.024194
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006211,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.018634,0.000000,0.012422,0.006211,0.000000,0.012422


Next i check top 5 venues for each of the Boroughs

In [23]:
num_top_venues = 5

for hood in df3_grouped['Borough']:
    print("----"+hood+"----")
    temp = df3_grouped[df3_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.06
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2           Restaurant  0.04
3  Japanese Restaurant  0.03
4                Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.06
2             Brewery  0.04
3                Café  0.04
4  Italian Restaurant  0.04


----West Toronto----
                venue  freq
0                Café  0.07
1                 Bar  0.06
2  Italian Restaurant  0.04
3          Restaurant  0.04
4         Coffee Shop  0.04




Then i have used a function defined to calculate 10 most common venues in a location 

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Borough_venues_sorted = pd.DataFrame(columns=columns)
Borough_venues_sorted['Borough'] = df3_grouped['Borough']

for ind in np.arange(df3_grouped.shape[0]):
    Borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df3_grouped.iloc[ind, :], num_top_venues)

Borough_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub
3,West Toronto,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore


Next i use kmeans method to cluster the Boroughs in 4 categories to check if the 4 categories are similar to the 4 Boroughs

In [26]:
# set number of clusters
kclusters =4

df3_grouped_clustering = df3_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df3_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 1], dtype=int32)

In [27]:
# add clustering labels
Borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df3_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df3_merged = df3_merged.join(Borough_venues_sorted.set_index('Borough'), on='Borough')

df3_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub


Next i create a map to add the 4 clusters along with the labels on the Toronto map.
As seen in the map the clusters are same as the 4 boroughs

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df3_merged['Latitude'], df3_merged['Longitude'], df3_merged['Borough'], df3_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now i explore the 4 clusters made to define each cluster by their type of venue 

Cluster 1: In this cluster there seem to be a number of cafes and looks like a good place to hang out in the evening

In [34]:
df3_merged.loc[df3_merged['Cluster Labels'] == 0, df3_merged.columns[[1] + list(range(2, df3_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
19,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
20,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
21,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
23,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
24,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
26,Central Toronto,Davisville,43.704324,-79.388790,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
29,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store
31,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,0,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Restaurant,Pub,Gym,Clothing Store


Cluster 2: In this cluster there are some restraunts, bars and cafes, thus this seems to be a good place for
heading during the night

In [35]:
df3_merged.loc[df3_merged['Cluster Labels'] == 1, df3_merged.columns[[1] + list(range(2, df3_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore
11,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore
14,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore
22,West Toronto,"High Park, The Junction South",43.661608,-79.464763,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore
25,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore
28,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,1,Café,Bar,Italian Restaurant,Restaurant,Coffee Shop,Bakery,Breakfast Spot,Park,Music Venue,Bookstore


Cluster 3: This cluster seems to have a mix of cafes and restraunts

In [36]:
df3_merged.loc[df3_merged['Cluster Labels'] == 2, df3_merged.columns[[1] + list(range(2, df3_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub
12,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub
15,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub
17,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub
38,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2,Greek Restaurant,Coffee Shop,Café,Brewery,Italian Restaurant,Pizza Place,Park,Restaurant,Ice Cream Shop,Pub


Cluster 4: This cluster seems to have a variety of food eatries and seems to be a good place to explore diffrent places to eat out

In [37]:
df3_merged.loc[df3_merged['Cluster Labels'] == 3, df3_merged.columns[[1] + list(range(2, df3_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
5,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
7,Downtown Toronto,Christie,43.669542,-79.422564,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
13,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Park,Seafood Restaurant
